In [1]:
!pip install langchain langchain-community langchain_groq chromadb gradio

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_sdk-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_api-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.58b0-py3-none-any.whl.metadata (2.4 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.7 MB/s  0:00:00 eta 0:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.

In [7]:
!pip install -U langchain


In [11]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [12]:
anime = pd.read_csv('anime_with_synopsis.csv')
anime.head()


,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [13]:
anime.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


In [16]:
anime['combine_df'] = anime.apply(lambda row:f"Title: {row['Name']}. Overview:{row['sypnopsis']} Genres:{row['Genres']}",axis=1)

In [18]:
anime['combine_df'].head()

0    Title: Cowboy Bebop. Overview:In the year 2071...
1    Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2    Title: Trigun. Overview:Vash the Stampede is t...
3    Title: Witch Hunter Robin. Overview:ches are i...
4    Title: Bouken Ou Beet. Overview:It is the dark...
Name: combine_df, dtype: object

In [20]:
anime[['combine_df']].to_csv('anime_updated.csv', index=False)

In [21]:
df = pd.read_csv('anime_updated.csv')
df.head()

,combine_df
0,Title: Cowboy Bebop. Overview:In the year 2071...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview:Vash the Stampede is t...
3,Title: Witch Hunter Robin. Overview:ches are i...
4,Title: Bouken Ou Beet. Overview:It is the dark...


In [27]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 719.1 kB/s  0:00:16m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 3.7 MB/s  0:01:56m0:00:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.0 MB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 1.4 MB/s  0:00:18m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [sentence-transformers]ence-transformers]


In [36]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

# --- PATCH: Safe embedding class that doesn't use numpy ---
class TorchSafeEmbeddings(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        # Encode directly to tensors to avoid numpy dependency
        embeddings = self.client.encode(texts, convert_to_numpy=False, convert_to_tensor=True)
        return [e.detach().cpu().tolist() for e in embeddings]

    def embed_query(self, text):
        embedding = self.client.encode(text, convert_to_numpy=False, convert_to_tensor=True)
        return embedding.detach().cpu().tolist()

# --- Load and split data ---
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# --- Use patched embeddings ---
embeddings = TorchSafeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- Create vector store (Chroma) ---
docssearch = Chroma.from_documents(texts, embeddings)

print("✅ Chroma vector store created successfully!")


✅ Chroma vector store created successfully!


In [76]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import torch

# --- PATCH: Avoid numpy dependency ---
class TorchSafeEmbeddings(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        embeddings = self.client.encode(texts, convert_to_numpy=False, convert_to_tensor=True)
        return [e.detach().cpu().tolist() for e in embeddings]

    def embed_query(self, text):
        embedding = self.client.encode(text, convert_to_numpy=False, convert_to_tensor=True)
        return embedding.detach().cpu().tolist()

# --- Load and split your CSV ---
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# --- Use the patched embedding class ---
embeddings = TorchSafeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- Initialize your LLM (Groq) ---
llm = ChatGroq(
    api_key="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",
    model="llama-3.3-70b-versatile",
    temperature=0
)

# --- Build your Chroma store ---
docssearch = Chroma.from_documents(texts, embeddings)

print("✅ Chroma vector store created successfully!")


✅ Chroma vector store created successfully!


In [77]:
#querying vector DB store for movie recomm
query = 'I am looking for an animated comedy movie.what could you suggest to me?'
docs = docssearch.similarity_search(query,k=1)
docs

[Document(metadata={'row': 216, 'source': 'anime_updated.csv'}, page_content='combine_df: Title: Ichigo 100%. Overview:One day, Manaka Junpei walks to the roof of his school and encounters a beautiful girl falling down from above him and accidentally exposing her strawberry panties. The embarrassed girl runs away before Junpei can find out whom she is. He wishes to become a filmmaker, and this whole experience seemed like it would make the perfect scene in a movie. And so he goes on a search for the girl with the strawberry panties in order to reenact it all on film. But he will soon discover that finding that one girl will not be that easy... (Source: ANN) Genres:Comedy, Ecchi, Harem, Romance, Shounen')]

In [81]:
#using QA Retrival for movie recomm
qa = RetrievalQA.from_chain_type(llm,chain_type = 'stuff',retriever = docssearch.as_retriever(),
                           return_source_documents = True)
qa

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1265178e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x126517550>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_ap

In [82]:
query = "Can you recommend a romance anime with a unique storyline?"
result = qa({"query":query })
result

{'query': 'Can you recommend a romance anime with a unique storyline?',
 'result': 'Based on the information I have, I can recommend "Aa! Megami-sama! (TV)". It has a unique storyline where a human, Keiichi Morisato, accidentally summons a goddess, Belldandy, through the Goddess Help Hotline, and his joke wish for her to stay with him forever is granted. The anime explores their blossoming relationship as they navigate their awkward and uncomfortable interactions, leading to an unexpected romance. The combination of comedy, supernatural, magic, and romance elements makes for an interesting and unique storyline.',
 'source_documents': [Document(metadata={'source': 'anime_updated.csv', 'row': 31}, page_content='combine_df: Title: Aa! Megami-sama! (TV). Overview:In a world where humans can have their wish granted via the Goddess Help Hotline, a human, Keiichi Morisato, summons the Goddess Belldandy by accident and jokes that she should stay with him forever. Unfortunately for him, his "wi

In [89]:
#prompt engineering
from langchain.prompts import PromptTemplate
template = """You are a movie recommender system that help users to find anime that match their preferences.
use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question:{question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables = ['context','question']
)
chain_type_kwargs = {'prompt':PROMPT}
qa = RetrievalQA.from_chain_type(llm,chain_type = 'stuff',retriever = docssearch.as_retriever(),
                           return_source_documents = True,
                                chain_type_kwargs = chain_type_kwargs)
#query = "Can you recommend a romance anime with a unique storyline?"
query = "Give me a list of comedy animes that are lighthearted and fun."
result = qa({"query":query })

print(result['result'])

Based on your request, I'd like to recommend the following three comedy anime that are lighthearted and fun:

1. **Toradora!**: This anime is a romantic comedy that follows the story of two high school students, Ryuuji and Taiga, who become entangled in a complicated relationship. With its witty dialogue and humorous characters, Toradora! is a lighthearted and entertaining watch that explores themes of love, friendship, and self-discovery.
2. **The Devil is a Part-Timer!**: This anime is a hilarious and quirky comedy that tells the story of Satan, the ruler of Hell, who is forced to work in a fast-food restaurant in modern-day Japan after being defeated by a hero. With its absurd premise and comedic characters, The Devil is a Part-Timer! is a fun and lighthearted anime that pokes fun at traditional fantasy tropes.
3. **Hataraku Saibou** (Cells at Work!): This anime is a comedic and educational series that personifies the cells in the human body, following the adventures of a red blood 

In [90]:
#second template
from langchain.prompts import PromptTemplate
template_prefix = """You are a movie recommender system that help users to find anime that match their preferences.
use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
            Age:{age}
            Gender:{gender}"""
template_suffix = """Question:{question}
Your response:"""

user_info = user_info.format(age=20, gender='female')
COMBINED_PROMPT  = template_prefix + '\n' + user_info + '\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find anime that match their preferences.
use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
            Age:20
            Gender:female
Question:{question}
Your response:


In [92]:

PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables = ['context','question']
)
chain_type_kwargs = {'prompt':PROMPT}
qa = RetrievalQA.from_chain_type(llm,chain_type = 'stuff',retriever = docssearch.as_retriever(),
                           return_source_documents = True,
                                chain_type_kwargs = chain_type_kwargs)
query = "Give me a list of comedy animes that are lighthearted and fun."
result = qa({"query":query })

print(result['result'])

Based on your preferences, I'd like to recommend the following three comedy anime shows that are lighthearted and fun:

1. **Toradora!**: This anime is a romantic comedy that follows the story of two high school students, Ryuuji and Taiga, as they navigate their relationships and personal growth. With its witty dialogue and humorous characters, Toradora! is a great choice for those looking for a lighthearted and entertaining anime.
2. **Ouran High School Host Club**: This anime is a comedy series that takes place in a prestigious high school, where a girl named Haruhi accidentally breaks a valuable vase and is forced to work for the school's host club to pay off her debt. With its hilarious characters, comedic misunderstandings, and heartwarming moments, Ouran High School Host Club is a fun and lighthearted anime that's perfect for a female audience.
3. **Hidamari Sketch**: This anime is a slice-of-life comedy that follows the daily lives of a group of high school girls living in a dor

In [126]:
import gradio as gr
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

#load anime dataset
anime = pd.read_csv('anime_with_synopsis.csv')
anime = anime.dropna()
anime['combine_df'] = anime.apply(lambda row:f"Title: {row['Name']}. Overview:{row['sypnopsis']} Genres:{row['Genres']}",axis=1)

#save updated dataset
anime[['combine_df']].to_csv('anime_updated.csv',index=False)
loader=CSVLoader(file_path="anime_updated.csv")
data = loader.load()

#text splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts=text_splitter.split_documents(data)

#create embedding
# --- Use the patched embedding class ---
embeddings = TorchSafeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- Initialize your LLM (Groq) ---
llm = ChatGroq(
    api_key="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",
    model="llama-3.3-70b-versatile",
    temperature=0
)
#custom prompt template
template = """You are a movie recommender system that help users to find anime that match their preferences.
use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question:{question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables = ['context','question']
)
chain_type_kwargs = {'prompt':PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,chain_type = 'stuff',retriever = docssearch.as_retriever(),
                           return_source_documents = True,
                                chain_type_kwargs = chain_type_kwargs)
import gradio as gr


# --- Recommendation function ---
def recommend_anime(query):
    try:
        result = qa({"query": query})
        output = result.get("result", "Sorry, no recommendations found.")
        return output.strip()
    except Exception as e:
        return f"Error: {e}"

# --- Your image URL ---
image_url = "https://cdn.theanimegallery.com/theanimegallery/f1cf4d6b-04cc-4723-ba40-74d52b063717-cute-wallpaper-anime.webp"  # Replace with your actual URL

# --- Custom CSS for background and centering ---
custom_css = f"""
.gradio-container {{
    background-image: url("{image_url}");
    background-size: cover;
    background-position: center;
    min-height: 100vh;
    font-family: 'Poppins', sans-serif;
}}

h1 {{
    color: #ffffff;
    text-shadow: 2px 2px 6px rgba(0,0,0,0.5);
    text-align: center;
}}


/* 🌸 Transparent component boxes */
.gr-box {{
   
    max-width: 65%;
    margin-left:2%;
}}

/* 🌸 Remove Gradio internal gray wrapper */
div.svelte-1vd8eap {{
    background: transparent !important;  /* remove gray background */
    box-shadow: none !important;         /* remove shadow */
    border: none !important;             /* remove borders */
}}

"""

# --- Gradio Blocks layout ---
with gr.Blocks(css=custom_css, title="Anime Movie Recommender") as iface:
    gr.Markdown("## 🌸 Anime Movie Recommender 🌸", elem_classes=["gr-box"])
    
    preference_box = gr.Textbox(
        label="🎬 Enter your anime preference",
        placeholder="e.g., Lighthearted romantic comedy with a fun storyline...",
        lines=2,
        elem_classes=["gr-box"]
    )
    
    description_box = gr.Textbox(
        value="Type the kind of anime you want to watch (romance, action, comedy, fantasy, etc.) and get 3 personalized recommendations below.",
        label="ℹ️ Description",
        lines=2,
        interactive=False,
        elem_classes=["gr-box"]
    )
    
    submit_btn = gr.Button("Get Recommendations", elem_classes=["gr-box"])
    
    recommend_box = gr.Textbox(
        label="✨ Recommended Anime",
        lines=8,
        interactive=False,
        elem_classes=["gr-box"]
    )
    
    
    
    # Connect button to function
    submit_btn.click(
        fn=recommend_anime,
        inputs=preference_box,
        outputs=recommend_box
    )

# --- Launch the app ---
if __name__ == "__main__":
    iface.launch()

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 34354724-9153-4234-984c-5ded8470dd88)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.
